##What are the characteristics of users who convert (behavioural and demographic)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, recall_score, precision_recall_curve, precision_score, roc_auc_score, roc_curve, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, precision_recall_curve
from sklearn.pipeline import Pipeline


In [2]:
df = pd.read_csv('/Users/youziya/Downloads/conversion_data.csv')

In [3]:
df.head()

,Unnamed: 0,visitor_id,visitor_location,marketing_channel,gender,product_views_Mon,product_views_Tue,product_views_Wed,product_views_Thu,product_views_Fri,...,time_on_site_Tue,time_on_site_Wed,time_on_site_Thu,time_on_site_Fri,pages_visited_Mon,pages_visited_Tue,pages_visited_Wed,pages_visited_Thu,pages_visited_Fri,converted_Fri
0,0,1,North America,youtube,M,4,5,2,1,2,...,33,23,46,57,4,5,5,4,6,1
1,1,2,South America,organic,F,2,5,0,1,5,...,28,39,24,51,5,6,7,6,2,0
2,2,3,Australia,instagram,M,3,4,0,-3,5,...,34,22,34,31,1,3,3,2,3,1
3,3,4,North America,facebook,M,1,0,0,5,0,...,26,43,22,39,3,4,7,7,1,0
4,4,5,Asia,instagram,F,1,5,2,2,3,...,54,30,47,57,6,6,5,2,4,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         2000 non-null   int64  
 1   visitor_id         2000 non-null   int64  
 2   visitor_location   2000 non-null   object 
 3   marketing_channel  2000 non-null   object 
 4   gender             2000 non-null   object 
 5   product_views_Mon  2000 non-null   int64  
 6   product_views_Tue  2000 non-null   int64  
 7   product_views_Wed  2000 non-null   int64  
 8   product_views_Thu  2000 non-null   int64  
 9   product_views_Fri  2000 non-null   int64  
 10  avg_rating_Mon     2000 non-null   float64
 11  avg_rating_Tue     2000 non-null   float64
 12  avg_rating_Wed     2000 non-null   float64
 13  avg_rating_Thu     2000 non-null   float64
 14  avg_rating_Fri     2000 non-null   float64
 15  avg_price_Mon      2000 non-null   float64
 16  avg_price_Tue      2000 

In [6]:
df.describe(include='all')

,Unnamed: 0,visitor_id,visitor_location,marketing_channel,gender,product_views_Mon,product_views_Tue,product_views_Wed,product_views_Thu,product_views_Fri,...,time_on_site_Tue,time_on_site_Wed,time_on_site_Thu,time_on_site_Fri,pages_visited_Mon,pages_visited_Tue,pages_visited_Wed,pages_visited_Thu,pages_visited_Fri,converted_Fri
count,2000.000000,2000.000000,2000,2000,2000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
unique,NaN,NaN,5,5,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,South America,instagram,F,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,433,431,1093,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,999.500000,1000.500000,NaN,NaN,NaN,2.541500,2.440500,2.441000,2.256500,2.354000,...,39.776500,40.155500,40.417000,40.028500,3.946000,4.071000,4.092500,3.921000,4.089500,0.372000
std,577.494589,577.494589,NaN,NaN,NaN,1.704914,1.715206,2.343345,2.370974,2.335682,...,11.855624,11.757399,11.892638,11.664207,2.010001,2.012209,2.002985,2.023316,2.026954,0.483459
min,0.000000,1.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000,-4.000000,-3.000000,...,20.000000,20.000000,20.000000,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,499.750000,500.750000,NaN,NaN,NaN,1.000000,1.000000,0.000000,0.000000,0.000000,...,29.000000,30.000000,30.000000,30.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0.000000
50%,999.500000,1000.500000,NaN,NaN,NaN,3.000000,2.000000,2.000000,2.000000,2.000000,...,40.000000,40.000000,40.500000,40.000000,4.000000,4.000000,4.000000,4.000000,4.000000,0.000000
75%,1499.250000,1500.250000,NaN,NaN,NaN,4.000000,4.000000,4.000000,5.000000,5.000000,...,50.000000,50.000000,51.000000,50.000000,6.000000,6.000000,6.000000,6.000000,6.000000,1.000000


##Data cleaning
drop null value and duplicates value also outliers

In [ ]:
df2= df.drop(['Unnamed'], axis=0, inplace=True)

In [ ]:
df2.isnull().sum()

In [ ]:
df2 = df2.dropna()

In [ ]:
dup = df2[df2.duplicated()]
dup

In [ ]:
df.isnull().sum()

In [ ]:
df2.drop(['description', 'steps', 'submitted'], axis=1, inplace=True)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(15,5))
sns.histplot(df2['Age'], ax=ax1)
sns.histplot(df2['Work_Experience'], ax=ax2)
sns.histplot(df2['Family_Size'], ax=ax3)

In [ ]:
numerical_columns = ['minutes', 'n_steps', 'n_ingredients', 'Calories', 'Total_fat', 'Sugar', 'Sodium', 'Protein', 'Saturated_fat', 'Carbs']

In [ ]:
# plot the histogram fo reach numerical columns so we can better make decisions on dropping outliers

# Set the number of bins for the histogram
num_bins = 20

# Plot histograms for each numerical column and see their skewness and tailness visually, added alpha feature here as not adding this feature will help the graphs display distribution precisely.
for column in numerical_columns:
    plt.figure(figsize=(8, 6))  # Adjust the figure size as per your preference
    plt.hist(df2[column], bins=num_bins, edgecolor='black', alpha=0.7)  # Added transparency
    plt.title(f'Histogram of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.show()

In [ ]:
descriptive_stats = df2.describe()
print(descriptive_stats)

Feature enginnering to convert object into dummy variables

In [25]:
df2['visitor_location'].unique()

array(['North America', 'South America', 'Australia', 'Asia', 'Europe'],
      dtype=object)

In [ ]:
location_mapping = {1:'North America', 2:'South America', 3:'Australia', 4:'Asia', 5:'Europe'}
df2['visitor_location'] = df2['visitor_location'].map(location_mapping)

In [26]:
df2['marketing_channel'].unique()

array(['youtube', 'organic', 'instagram', 'facebook', 'google'],
      dtype=object)

In [ ]:
channel_mapping = {1:'youtube', 2:'organic', 3:'instagram', 4:'facebook', 5:'google'}
df2['marketing_channel'] = df2['marketing_channel'].map(channel_mappin)

In [17]:
location_dummies = pd.get_dummies(df2['visitor_location'], dtype=int)
channel_dummies = pd.get_dummies(df2['marketing_channel'], dtype=int)
gender_dummies = pd.get_dummies(df2['gender'], dtype=int)

In [18]:
location_dummies.head()

,F,M
0,0,1
1,1,0
2,0,1
3,0,1
4,1,0


In [20]:
df2 = pd.concat([df2,location_dummies], axis=1)
df2 = pd.concat([df2,channel_dummies], axis=1)
df2 = pd.concat([df2,gender_dummies], axis=1)

In [ ]:
df2.drop(['visitor_location', 'marketing_channel', 'gender'], axis=1, inplace=True)

In [21]:
df2.head()

,Unnamed: 0,visitor_id,visitor_location,marketing_channel,gender,product_views_Mon,product_views_Tue,product_views_Wed,product_views_Thu,product_views_Fri,...,Europe,North America,South America,facebook,google,instagram,organic,youtube,F,M
0,0,1,North America,youtube,M,4,5,2,1,2,...,0,1,0,0,0,0,0,1,0,1
1,1,2,South America,organic,F,2,5,0,1,5,...,0,0,1,0,0,0,1,0,1,0
2,2,3,Australia,instagram,M,3,4,0,-3,5,...,0,0,0,0,0,1,0,0,0,1
3,3,4,North America,facebook,M,1,0,0,5,0,...,0,1,0,1,0,0,0,0,0,1
4,4,5,Asia,instagram,F,1,5,2,2,3,...,0,0,0,0,0,1,0,0,1,0
